In [23]:
import pandas as pd
from nltk.corpus import stopwords

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn import metrics
from sklearn.metrics import accuracy_score

import string

In [24]:
df_dataset = pd.read_csv('./output/US_cleaned_data_2018')
df_dataset.head()

,Unnamed: 0,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,video_error_or_removed,description
0,0,2kyS6SvSYSE,17.11.14,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,People & Blogs,17.11.13,['SHANtell martin'],748374,57527,2966,15954,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1,1ZAPwfrtAFY,17.11.14,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,Entertainment,17.11.13,"['last week tonight trump presidency', '""last ...",2418783,97185,6146,12703,False,"One year after the presidential election, John..."
2,2,5qpjK5DgCt4,17.11.14,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,Comedy,17.11.12,"['racist superman', '""rudy""', '""mancuso""', '""k...",3191434,146033,5339,8181,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,3,puqaWrEC7tY,17.11.14,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,Entertainment,17.11.13,"['rhett and link', '""gmm""', '""good mythical mo...",343168,10172,666,2146,False,Today we find out if Link is a Nickelback amat...
4,4,d380meD0W0M,17.11.14,I Dare You: GOING BALD!?,nigahiga,Entertainment,17.11.12,"['ryan', '""higa""', '""higatv""', '""nigahiga""', '...",2095731,132235,1989,17518,False,I know it's been a while since we did this sho...


In [25]:
# https://adataanalyst.com/scikit-learn/countvectorizer-sklearn-example/
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    3. Return String
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    list_words = nopunc.split()
    
    sentence = ' '.join(list_words)
    
    return sentence.lower()

In [26]:
df_titles_info = pd.DataFrame(df_dataset['title'])
df_titles_info['filter_title'] = df_titles_info['title'].apply(text_process)

df_titles_info['description'] = df_dataset['description'].fillna('')
df_titles_info['description'] = df_titles_info['description'].apply(text_process)

df_titles_info['category_id'] = df_dataset['category_id']

df_titles_info

,title,filter_title,description,category_id
0,WE WANT TO TALK ABOUT OUR MARRIAGE,we want to talk about our marriage,shantells channel httpswwwyoutubecomshantellma...,People & Blogs
1,The Trump Presidency: Last Week Tonight with J...,the trump presidency last week tonight with jo...,one year after the presidential election john ...,Entertainment
2,"Racist Superman | Rudy Mancuso, King Bach & Le...",racist superman rudy mancuso king bach lele pons,watch my previous video ▶ nnsubscribe ► httpsw...,Comedy
3,Nickelback Lyrics: Real or Fake?,nickelback lyrics real or fake,today we find out if link is a nickelback amat...,Entertainment
4,I Dare You: GOING BALD!?,i dare you going bald,i know its been a while since we did this show...,Entertainment
...,...,...,...,...
40944,The Cat Who Caught the Laser,the cat who caught the laser,the cat who caught the laser aarons animals,Pets & Animals
40945,True Facts : Ant Mutualism,true facts ant mutualism,,People & Blogs
40946,I GAVE SAFIYA NYGAARD A PERFECT HAIR MAKEOVER ...,i gave safiya nygaard a perfect hair makeover ...,i had so much fun transforming safiyas hair in...,Entertainment
40947,How Black Panther Should Have Ended,how black panther should have ended,how black panther should have endedwatch more ...,Film & Animation


In [27]:
df_x = df_titles_info['filter_title']
df_y = df_titles_info['category_id']

df_titles_info.to_csv('./output/US_count_vectorizer_dataset.csv', index=False)

target_names = list(df_titles_info['category_id'].unique())

In [28]:
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, random_state=117)

In [29]:
count_vectorizer = CountVectorizer()
train_count_vector = count_vectorizer.fit_transform(x_train)
test_count_vector = count_vectorizer.transform(x_test)
print(train_count_vector.shape)
# print(count_vectorizer.vocabulary_)
print(train_count_vector.toarray())

(32759, 10574)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [30]:
tfidf_vectorizer = TfidfTransformer()
x_trained_tfidf_vector = tfidf_vectorizer.fit_transform(train_count_vector)
x_test_tfidf_vector = tfidf_vectorizer.transform(test_count_vector)
print(x_trained_tfidf_vector.shape)
# print(count_vectorizer.vocabulary_)
print(x_trained_tfidf_vector.toarray())

(32759, 10574)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [31]:
clf_tfidf = MultinomialNB().fit(x_trained_tfidf_vector, y_train)

In [32]:
pred_tfidf = clf_tfidf.predict(x_test_tfidf_vector)
print("Accuracy: ", accuracy_score(y_test, pred_tfidf))
print(metrics.classification_report(y_test, pred_tfidf, target_names=target_names))

Accuracy:  0.860927960927961
                       precision    recall  f1-score   support

       People & Blogs       1.00      0.20      0.33        91
        Entertainment       0.92      0.77      0.84       664
               Comedy       0.98      0.79      0.88       329
 Science & Technology       0.72      0.97      0.82      2016
     Film & Animation       0.99      0.75      0.86       495
      News & Politics       1.00      0.59      0.74       149
               Sports       0.89      0.91      0.90       823
                Music       0.91      0.96      0.93      1245
       Pets & Animals       0.96      0.85      0.90       523
            Education       0.00      0.00      0.00        10
        Howto & Style       0.96      0.72      0.82       660
     Autos & Vehicles       0.99      0.77      0.87       186
      Travel & Events       0.91      0.84      0.87       461
               Gaming       1.00      0.57      0.73        14
Nonprofits & Activism    

/home/loc/anaconda3/envs/yt-trends/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
clf_count = MultinomialNB().fit(train_count_vector, y_train)
pred_count = clf_count.predict(test_count_vector)

In [34]:
print("Accuracy: ", accuracy_score(y_test, pred_count))
print(metrics.classification_report(y_test, pred_count, target_names=target_names))

Accuracy:  0.8996336996336997
                       precision    recall  f1-score   support

       People & Blogs       0.93      0.75      0.83        91
        Entertainment       0.91      0.82      0.86       664
               Comedy       0.91      0.95      0.93       329
 Science & Technology       0.89      0.89      0.89      2016
     Film & Animation       0.93      0.88      0.90       495
      News & Politics       0.96      0.89      0.92       149
               Sports       0.85      0.92      0.89       823
                Music       0.92      0.96      0.94      1245
       Pets & Animals       0.92      0.93      0.92       523
            Education       1.00      0.20      0.33        10
        Howto & Style       0.91      0.83      0.87       660
     Autos & Vehicles       0.94      0.89      0.91       186
      Travel & Events       0.87      0.92      0.89       461
               Gaming       1.00      0.86      0.92        14
Nonprofits & Activism   